In [150]:
# Code adapted from the work of Richard Kuo shared on Kaggle
# The Python code has been published at: https://www.kaggle.com/rkuo2000/financial-statement-analysis
# import libraries
from datetime import datetime
import lxml
from lxml import html
import requests
import numpy as np
import pandas as pd

In [151]:
# input a stock symbol
symbolMRU = 'MRU.TO'

In [152]:
# Yahoo Finance links
urlMRU_bs = 'https://finance.yahoo.com/quote/' + symbolMRU + '/balance-sheet?p=' + symbolMRU
urlMRU_is = 'https://finance.yahoo.com/quote/' + symbolMRU + '/financials?p=' + symbolMRU
urlMRU_cf = 'https://finance.yahoo.com/quote/' + symbolMRU + '/cash-flow?p='+ symbolMRU

In [153]:
# Set up the request headers that we're going to use, to simulate a request by the Chrome browser. 
# Simulating a request from a browser is generally good practice when building a scraper
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Pragma': 'no-cache',
    'Referrer': 'https://google.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'
}

In [154]:
def get_table(url):
    # Fetch the page that we're going to parse, using the request headers defined above
    page = requests.get(url, headers)

    # Parse the page with LXML, so that we can start doing some XPATH queries
    # to extract the data that we want
    tree = html.fromstring(page.content)

    # Smoke test that we fetched the page by fetching and displaying the H1 element
    tree.xpath("//h1/text()")
    table_rows = tree.xpath("//div[contains(@class, 'D(tbr)')]")

    # Ensure that some table rows are found; if none are found, then it's possible
    # that Yahoo Finance has changed their page layout, or have detected
    # that you're scraping the page.
    assert len(table_rows) > 0

    parsed_rows = []

    for table_row in table_rows:
        parsed_row = []
        el = table_row.xpath("./div")

        none_count = 0

        for rs in el:
            try:
                (text,) = rs.xpath('.//span/text()[1]')
                parsed_row.append(text)
            except ValueError:
                parsed_row.append(np.NaN)
                none_count += 1

        if (none_count < 4):
            parsed_rows.append(parsed_row)

    df = pd.DataFrame(parsed_rows)
    df_org = df
    
    df = pd.DataFrame(parsed_rows)
    df = df.set_index(0) # Set the index to the first column: 'Period Ending'.
    df = df.transpose() # Transpose the DataFrame, so that our header contains the account names

    # Rename the "Breakdown" column to "Date"
    cols = list(df.columns)
    cols[0] = 'Date'
    df = df.set_axis(cols, axis='columns', inplace=False)
    df_rot = df
    return df_org, df_rot

In [155]:
# get Balance Sheet 
urlBSMRU = 'https://raw.githubusercontent.com/hadrienpierre/IND8122-CS1/main/Annual_BalanceSheet_MRU_imp.csv'
BSMRU_orginal= pd.read_csv (urlBSMRU, index_col=0)
BSMRU_transpose = BSMRU_orginal.transpose() 

# get Income Statement 
ISMRU_orginal, ISMRU_transpose = get_table(urlMRU_is)

# get Cash Flow
CFMRU_orginal, CFMRU_transpose = get_table(urlMRU_cf)

In [156]:
BSMRU_orginal

,9/30/2019,9/30/2018,9/30/2017,9/30/2016,9/30/2015
Date fin exercice,,,,,
Total de l'actif,"1,107,390","1,092,220","605,070","560,610","538,710"
Actifs courants,"208,830","191,680","135,630","1,193","117,290"
Trésorerie équivalents de trésorerie et placements à court terme,27340,22690,14890,2750,2150
Trésorerie et équivalents de trésorerie,27340,22690,14890,2750,2150
En espèces,—,—,—,—,—
Équivalents de trésorerie,—,—,—,—,—
Placements à court terme,—,—,—,—,—
Débiteurs,65570,55870,33180,31830,30370
Comptes clients,61120,53810,31370,30640,29060


In [157]:
ISMRU_orginal

,0,1,2,3,4
0,Breakdown,ttm,9/30/2019,9/30/2018,9/30/2017
1,Total Revenue,"17,712,800","16,767,500","14,383,400","13,175,300"
2,Cost of Revenue,"14,185,600","13,438,800","11,556,500","10,579,600"
3,Gross Profit,"3,527,200","3,328,700","2,826,900","2,595,700"
4,Operating Expense,"2,330,500","2,263,600","2,006,500","1,823,500"
5,Operating Income,"1,196,700","1,065,100","820,400","772,200"
6,Net Non Operating Interest Income Expense,"-129,400","-103,800","-80,200","-63,900"
7,Other Income Expense,"-7,500","7,900","1,336,500","93,500"
8,Pretax Income,"1,059,800","969,200","2,076,700","801,800"
9,Tax Provision,"282,500","254,800","358,200","193,400"


In [158]:
CFMRU_orginal

,0,1,2,3,4
0,Breakdown,ttm,9/30/2019,9/30/2018,9/30/2017
1,Operating Cash Flow,"1,184,500","687,700","750,400","696,200"
2,Investing Cash Flow,"-408,300","-308,500","-1,677,500","-333,000"
3,Financing Cash Flow,"-672,900","-332,700","1,005,100","-241,800"
4,End Cash Position,"366,600","273,400","226,900","148,900"
5,Capital Expenditure,"-460,000","-396,300","-317,400","-368,900"
6,Issuance of Capital Stock,"14,600","24,000","8,800","10,700"
7,Issuance of Debt,"413,400","46,600","2,168,800","737,700"
8,Repayment of Debt,"-633,100","-54,000","-996,200","-537,300"
9,Repurchase of Capital Stock,"-214,600","-151,500","-10,200","-309,500"


In [159]:
# Balance Sheet transpose table
BSMRU_transpose

Date fin exercice,Total de l'actif,Actifs courants,Trésorerie équivalents de trésorerie et placements à court terme,Trésorerie et équivalents de trésorerie,En espèces,Équivalents de trésorerie,Placements à court terme,Débiteurs,Comptes clients,Comptes clients bruts,...,Actifs corporels nets,Fonds de roulement,Capital investi,Valeur comptable des immobilisations corporelles,Total de la dette,Dette nette,Actions émises,Nombre d’actions ordinaires,Nombre d’actions privilégiées,Nombre d’actions propres
9/30/2019,"1,107,390","208,830",27340,27340,—,—,—,65570,61120,—,...,—,—,—,—,—,—,—,—,—,—
9/30/2018,"1,092,220","191,680",22690,22690,—,—,—,55870,53810,—,...,—,—,—,—,—,—,—,—,—,—
9/30/2017,"605,070","135,630",14890,14890,—,—,—,33180,31370,—,...,—,—,—,—,—,—,—,—,—,—
9/30/2016,"560,610","1,193",2750,2750,—,—,—,31830,30640,—,...,—,—,—,—,—,—,—,—,—,—
9/30/2015,"538,710","117,290",2150,2150,—,—,—,30370,29060,—,...,—,—,—,—,—,—,—,—,—,—


### Ratio du Fond de Roulement = Actif Courant / Passif Courant

In [160]:
current_assets = BSMRU_transpose["Actifs courants"].str.replace(',', '').astype(int)
current_liabilities = BSMRU_transpose["Passifs courants"].str.replace(',', '').astype(int)
rfdr = current_assets / current_liabilities
BSMRU_analysis = pd.DataFrame(rfdr)
BSMRU_analysis.columns = ['Ratio Fond de Roulement']
BSMRU_analysis

,Ratio Fond de Roulement
9/30/2019,1.112218
9/30/2018,1.172570
9/30/2017,1.054748
9/30/2016,0.011176
9/30/2015,1.102557


## Levier Financier

### Endettement Total

In [161]:
capitaux_propres = BSMRU_transpose["Capitaux propres"].str.replace(',', '').astype(int)
total_assets = BSMRU_transpose["Total de l'actif"].str.replace(',', '').astype(int)
BSMRU_analysis['Endettement Total'] = (total_assets - capitaux_propres) / total_assets
BSMRU_analysis

,Ratio Fond de Roulement,Endettement Total
9/30/2019,1.112218,0.462231
9/30/2018,1.172570,0.483364
9/30/2017,1.054748,0.518882
9/30/2016,0.011176,0.521842
9/30/2015,1.102557,0.509309


### Dette/Fonds Propres

In [162]:
total_liabilities = BSMRU_transpose["Total du passif"].str.replace(',', '').astype(int)
BSMRU_analysis['Dettes-Fonds Propres'] = (total_assets - capitaux_propres) / total_assets
BSMRU_analysis

,Ratio Fond de Roulement,Endettement Total,Dettes-Fonds Propres
9/30/2019,1.112218,0.462231,0.462231
9/30/2018,1.172570,0.483364,0.483364
9/30/2017,1.054748,0.518882,0.518882
9/30/2016,0.011176,0.521842,0.521842
9/30/2015,1.102557,0.509309,0.509309


### Endettement à long terme
Ratio d'endettement à long terme = Passif à long terme / (Passif à long terme + Fonds Propres)

In [163]:
longterm_liabilities = BSMRU_transpose["Total des passifs non courants"].str.replace(',', '').astype(int)
BSMRU_analysis['Endettement Long Terme'] = (total_assets - capitaux_propres) / total_assets
BSMRU_analysis

,Ratio Fond de Roulement,Endettement Total,Dettes-Fonds Propres,Endettement Long Terme
9/30/2019,1.112218,0.462231,0.462231,0.462231
9/30/2018,1.172570,0.483364,0.483364,0.483364
9/30/2017,1.054748,0.518882,0.518882,0.518882
9/30/2016,0.011176,0.521842,0.521842,0.521842
9/30/2015,1.102557,0.509309,0.509309,0.509309


## Utilisation de l'actif

### Rotation des Stocks
Coûts des marchandises vendues / Stocks